### Libraries

In [481]:
import math
import numpy as np

# import pandas 
# import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go

### Constants

In [482]:
# @title Gauss-Laguerre constants
# Notation: (A_i, x_i)

GAUSS_LAGUERRE = [
    (0.458964, 0.222847),
    (0.417, 1.188932),
    (0.113373, 2.992736),
    (0.0103992, 5.775144),
    (0.000261017, 9.837467),
    (0.000000898548, 15.982874)
]

### PDF, CDF of Weibull distribution

In [483]:
# @title Weibull distribution function
# Uses Gauss-Laguerre quadratures
def f_Weibull (t, k, mu):
    denomArea = 0
    
    for node in GAUSS_LAGUERRE:
        denomArea += node[0] * (node[1]**(1/k))

    lbda = mu / denomArea

    return (k/lbda) * ((t/lbda)**(k-1)) * (math.e ** (-(t/lbda)**k))

In [484]:
# @title Integral of Weibull function
# Using Gauss-Laguerre quadratures
def cdf (t, k, mu):
    pdf_area = 0
    
    for node in GAUSS_LAGUERRE:
        pdf_area += node[0] * (math.e**node[1]) * f_Weibull(t + node[1], k, mu)

    return 1 - pdf_area

In [485]:
def plot(x05, x1, x2, y05, y1, y2, xtitle, ytitle):
    fig = go.Figure()
    fig.add_traces([
        go.Scatter(x=x05, y=y05, mode='lines', marker = {'color' : 'blue'}, name="k = 0.5"),
        go.Scatter(x=x1, y=y1, mode='lines', marker = {'color' : 'red'}, name="k = 1"),
        go.Scatter(x=x2, y=y2, mode='lines', marker = {'color' : 'magenta'}, name="k = 2")
    ])
    fig.update_layout(
        height=1080*0.5,
        width=1920*0.6,
        xaxis_title=xtitle,
        yaxis_title=ytitle
    )
    fig.show()

##### PDF

In [486]:
# @title CDF values
x, y05, y1, y2 = [], [], [], []

for t in range(1, 251, 1):
    t /= 100
    x.append(t)
    y05.append(f_Weibull(t, 0.5, 2))
    y1.append(f_Weibull(t, 1, 1))
    y2.append(f_Weibull(t, 2, math.sqrt(math.pi)/2))

plot(x, x, x, y05, y1, y2, "t", "alpha")

##### CDF evaluation and plotting

In [487]:
# @title CDF values
x, y05, y1, y2 = [], [], [], []

for t in range(1, 251, 1):
    t /= 100
    x.append(t)
    y05.append(cdf(t, 0.5, 2))
    y1.append(cdf(t, 1, 1))
    y2.append(cdf(t, 2, math.sqrt(math.pi)/2))

plot(x, x, x, y05, y1, y2, "t", "alpha")

### Survival function

##### S(t)

In [488]:
# @title Survival function
plot(x, x, x, [1-i for i in y05], [1-i for i in y1], [1-i for i in y2], "t", "alpha")

##### S^-1(alpha)

In [489]:
# @title Inverse survival function
plot([1-i for i in y05], [1-i for i in y1], [1-i for i in y2], x, x, x, "alpha", "t")

### Root-finding problem

In [490]:
# @title Inverse survivability
# solved as root-finding problem using bisection method
def bisection (a, b, alpha, mu, k):
    f = lambda t : 1 - cdf(t, k, mu) - alpha

    tol = 1.0e-9

    fa = f(a)
    fb = f(b)

    if fa == 0.0:
        return a
    
    if fb == 0.0:
        return b
    
    print("-> CDF(" + str(a) + ") - " + str(alpha) + " = " + str(fa))
    print("-> CDF(" + str(b) + ") - " + str(alpha) + " = " + str(fb))
    
    if np.sign(fa) == np.sign(fb):
        print("---> no root at: ", alpha)
        return None
    
    n = int (math.ceil (math.log(abs(b-a)/tol) / math.log(2.0)))

    print("---> iterations: ", n)

    for i in range(n):
        print("-> CDF(" + str(a) + ") - " + str(alpha) + " = " + str(fa))
        print("-> CDF(" + str(b) + ") - " + str(alpha) + " = " + str(fb))

        c = 0.5 * (a + b)
        fc = f(c)

        if fc == 0.0:
            print("---> returning c =", c)
            return c

        if np.sign(fa) != np.sign(fc):
            b = c
            fb = fc
        
        elif np.sign(fb) != np.sign(fc):
            a = c
            fa = fc

    return 0.5 * (a+b)

#### Evaluation

In [491]:
# Lists for values storing
x_values, y_t_05, y_t_1, y_t_2 = [], [], [], []

# Average life expectancy
mu = 78

In [492]:
# For alpha in (0,1) with 0.01 step
for alpha in range(12, 100, 1):
    alpha /= 100

    x_values.append(alpha)

    # y_t_05.append(bisection(0, 100, alpha, mu, 0.5))
    # y_t_1.append(bisection(0, 100, alpha, mu, 1))
    y_t_2.append(bisection(0, 100, alpha, mu, 2))

    # y_t_05.append(bisection(0, 2.5, alpha, 2, 0.5))
    # y_t_1.append(bisection(0, 2.5, alpha, 1, 1))
    # y_t_2.append(bisection(0, 2.5, alpha, math.sqrt(math.pi)/2, 2))

-> CDF(0) - 0.12 = -0.06843800785366394
-> CDF(100) - 0.12 = -1.0484024275503323e-05
---> no root at:  0.12
-> CDF(0) - 0.13 = -0.07843800785366395
-> CDF(100) - 0.13 = -0.010010484024275512
---> no root at:  0.13
-> CDF(0) - 0.14 = -0.08843800785366396
-> CDF(100) - 0.14 = -0.02001048402427552
---> no root at:  0.14
-> CDF(0) - 0.15 = -0.09843800785366394
-> CDF(100) - 0.15 = -0.030010484024275502
---> no root at:  0.15
-> CDF(0) - 0.16 = -0.10843800785366395
-> CDF(100) - 0.16 = -0.04001048402427551
---> no root at:  0.16
-> CDF(0) - 0.17 = -0.11843800785366396
-> CDF(100) - 0.17 = -0.05001048402427552
---> no root at:  0.17
-> CDF(0) - 0.18 = -0.12843800785366394
-> CDF(100) - 0.18 = -0.0600104840242755
---> no root at:  0.18
-> CDF(0) - 0.19 = -0.13843800785366395
-> CDF(100) - 0.19 = -0.07001048402427551
---> no root at:  0.19
-> CDF(0) - 0.2 = -0.14843800785366396
-> CDF(100) - 0.2 = -0.08001048402427552
---> no root at:  0.2
-> CDF(0) - 0.21 = -0.15843800785366394
-> CDF(100) - 

#### Plotting

In [493]:
# @title Inverse survivability plot
fig = go.Figure()
fig.add_traces([
    go.Scatter(x=x_values, y=y_t_05, mode='lines', marker = {'color' : 'blue'}, name="k = 0.5"),
    go.Scatter(x=x_values, y=y_t_1, mode='lines', marker = {'color' : 'red'}, name="k = 1"),
    go.Scatter(x=x_values, y=y_t_2, mode='lines', marker = {'color' : 'magenta'}, name="k = 2")
])
fig.update_layout(
    height=1080*0.5,
    width=1920*0.6,
    xaxis_title="Probability (alpha)",
    yaxis_title="Time (t)"
)
fig.show()

In [494]:
print(x_values[10:])
print(y_t_2[10:])

[0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.7, 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99]
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
